In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from results_aggregator import ResultsAggregator
from metrics_computer import EvalFilter

## Load data

In [ ]:
res_agg = ResultsAggregator()
exp_dir_eval=f"../generation/results/shifts/eval"
exp_ids = [0,]
df_res = res_agg.load_evaluation_results(exp_dir_eval, exp_ids)
classifiers = df_res['classifier'].unique()
df_delt = res_agg.compute_delta_quantity_to_ref(df_res, quantity_to_delta='CLIP_shift', new_col_name='delta_clip_shift', ref_col_name='var_1', groupby=['i_class', 'seed', 'exp_id','var_2'],negate=False)
df_delt = res_agg.compute_delta_quantity_to_ref(df_delt, quantity_to_delta='CLIP_class_shift', new_col_name='delta_clip_class_in_shift', ref_col_name='var_1', groupby=['i_class', 'seed', 'exp_id','var_2'],negate=False)
df_delt = res_agg.compute_delta_quantity_to_ref(df_delt, quantity_to_delta='CLIP_class', new_col_name='delta_clip_class', ref_col_name='var_1', groupby=['i_class', 'seed', 'exp_id','var_2',],negate=False)
df_delt.head(3)

# Filtering

In [ ]:
# Thresholds computed on the filtering dataset to remove 90% of the OOC samples.
quantile_ths = {'CLIP_class': 28.7394449, 'CLIP_ref': 0.805859378, 'CLIP_class_shift': 35.292848400000004, 'DINO_ref_no_head': 0.7811618779999999}


exp_id = 0
df_e = df_delt[df_delt['exp_id']==exp_id]
m_ic = EvalFilter.filter_k_out_of_n(df_e,quantile_ths=quantile_ths)
df_e = df_e[m_ic]
df_e = df_e.groupby(['exp_id','i_class','seed']).filter(lambda x: len(x) == 6)
print('Took only complete sliding trajectories. Remaining images:', len(df_e))
df_e.to_csv(f'../generation/results/shifts/filtered/eval_id_{exp_id:04d}.csv')